In [24]:
import numpy as np

# Wczytujemy dane
train = np.load("train.npz")
test = np.load("test.npz")

xs_train = train["xs"]
y_train = train["y"]
xs_test = test["xs"]

print("Wymiary xs_train:", xs_train.shape)
print("Wymiary y_train:", y_train.shape)
print("Wymiary xs_test:", xs_test.shape)

Wymiary xs_train: (2000, 20)
Wymiary y_train: (2000,)
Wymiary xs_test: (500, 20)


In [26]:
# Funkcja wyznaczająca cechy: [ile(0), ile(1), ile(00), ile(01), ile(10), ile(11)]
def wyznacz_cechy(xs):
    n = xs.shape[0]
    cechy = np.zeros((n, 6), dtype=float)

    for i in range(n):
        x = xs[i]

        # pojedyncze bity
        cechy[i, 0] = np.sum(x==0)
        cechy[i, 1] = np.sum(x==1)

        # pary nachodzące
        for j in range(len(x)-1):
            if x[j]==0 and x[j+1]==0:
                cechy[i, 2]+=1
            elif x[j]==0 and x[j+1]==1:
                cechy[i, 3]+=1
            elif x[j]==1 and x[j+1]==0:
                cechy[i, 4]+=1
            else:
                cechy[i, 5]+=1

    return cechy


# Wyznaczamy cechy dla zbioru treningowego i testowego
X_train = wyznacz_cechy(xs_train)
X_test = wyznacz_cechy(xs_test)


# Uczymy model (regresja liniowa – najmniejsze kwadraty)
wspolczynniki, _, _, _ = np.linalg.lstsq(X_train, y_train, rcond=None)


# Przewidujemy wartości dla zbioru testowego
y_test_pred = X_test @ wspolczynniki


MSE na zbiorze treningowym: 60.4144


In [27]:
# Zapisujemy rozwiązanie
assert y_test_pred.shape == (xs_test.shape[0],), "Nieprawidłowe wymiary zapisywanych wyników!"
np.savez("solution.npz", y=y_test_pred)
print("Zapisano plik 'solution.npz' z wynikami dla zbioru testowego.")

Zapisano plik 'solution.npz' z wynikami dla zbioru testowego.
